In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import missingno as msno
import seaborn as sns
import scipy.stats as st
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost as xg
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
from eli5.sklearn import PermutationImportance
from eli5 import show_weights
from sklearn.tree import DecisionTreeClassifier
from bayes_opt import BayesianOptimization

In [2]:
#Загрузим данные из файла.
df = pd.read_excel('/home/rik/Рабочий стол/МИФИ/учеба/мо/curse/data.xlsx', index_col=0)
df.head()

,"IC50, mM","CC50, mM",SI,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.239374,175.482382,28.125000,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,...,0,0,0,0,0,0,0,0,3,0
1,0.771831,5.402819,7.000000,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,...,0,0,0,0,0,0,0,0,3,0
2,223.808778,161.142320,0.720000,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,...,0,0,0,0,0,0,0,0,3,0
3,1.705624,107.855654,63.235294,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,...,0,0,0,0,0,0,0,0,4,0
4,107.131532,139.270991,1.300000,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,...,0,0,0,0,0,0,0,0,0,0


In [3]:
print("train.shape = {} rows, {} cols".format(*df.shape))

train.shape = 1001 rows, 213 cols


# Предобработка данных.

In [4]:
def transform(df): 
    discrete_feature = [feature for feature in df.columns if len(df[feature].unique())<25]
    count_discrete_feature = {}
    for i in tqdm(discrete_feature):
        cnt = 0
        for j in df[i]:
            if j != 0:
                cnt += 1
        count_discrete_feature[f'{i}'] = cnt
    count_discrete_feature = pd.Series(count_discrete_feature)
    df = df.drop(columns=count_discrete_feature[count_discrete_feature == 0].index.to_list())
    df = df.dropna()
    feature_del = ['BertzCT', 'Chi0', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v',
     'ExactMolWt', 'FpDensityMorgan1', 'FpDensityMorgan3', 'HeavyAtomCount', 'HeavyAtomMolWt', 'Kappa1', 'Kappa2',
     'Kappa3', 'LabuteASA', 'MaxAbsEStateIndex', 'MaxAbsPartialCharge', 'MolMR', 'MolWt', 'NHOHCount', 'NumAromaticCarbocycles',
     'NumHAcceptors', 'NumHeteroatoms', 'NumSaturatedCarbocycles', 'NumValenceElectrons', 'SMR_VSA4', 'SMR_VSA9',
     'SlogP_VSA6', 'TPSA', 'VSA_EState2', 'VSA_EState3', 'VSA_EState6', 'fr_Al_OH_noTert', 'fr_COO', 'fr_COO2',
     'fr_C_O_noCOO', 'fr_Nhpyrrole', 'fr_benzene', 'fr_nitro_arom_nonortho', 'fr_phenol', 'fr_phenol_noOrthoHbond',
    'MinAbsPartialCharge', 'Chi2n']
    df = df.drop(columns=feature_del)
   
    return df

In [5]:
df = transform(df)

100%|███████████████████████████████████████| 110/110 [00:00<00:00, 6009.97it/s]


In [6]:
val_median = df['IC50, mM'].median()

In [7]:
df['IC50, mM'] = df['IC50, mM'].apply(lambda x: 0 if x < val_median else 1)
df['IC50, mM'].head()

0    0
1    0
2    1
3    0
4    1
Name: IC50, mM, dtype: int64

## Обучение нескольких моделей, их сравнение. 

In [8]:
target = df['IC50, mM']
data = df.drop(['CC50, mM','IC50, mM','SI'], axis=1)

In [9]:
x_train, x_valid = train_test_split(
    data, train_size=0.8, random_state=1
)
y_train, y_valid = train_test_split(
    target, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))

x_train.shape = 798 rows, 144 cols
x_valid.shape = 200 rows, 144 cols


In [10]:
# Создадим модель дерева решений
tree = DecisionTreeClassifier(random_state=27)
# Создадим модель CatBoost
model = CatBoostClassifier(random_state=27)
# Создадим модель lightgbm
model_LGBM = LGBMClassifier(random_state=27)
# Создадим модель xgboost
xgb_r = xg.XGBClassifier() 
# обучение моделей
tree.fit(x_train, y_train)
model.fit(x_train, y_train, verbose=0)
model_LGBM.fit(x_train, y_train, verbose=0)
xgb_r.fit(x_train, y_train, verbose=0)
# предсказание ответов для тренеровочной выборки
y_pred_tree_train = tree.predict(x_train)
y_pred_сat_Boost_train = model.predict(x_train)
y_pred_LGBM_train = model_LGBM.predict(x_train)
y_pred_xgb_train = xgb_r.predict(x_train)
# предсказание ответов для тестовой выборки
y_pred_tree = tree.predict(x_valid)
y_pred_сat_Boost = model.predict(x_valid)
y_pred_LGBM = model_LGBM.predict(x_valid)
y_pred_xgb = xgb_r.predict(x_valid)

print("Train tree f1_score: ", f1_score(y_train, y_pred_tree_train))
print("Train tree accuracy_score: ", accuracy_score(y_train, y_pred_tree_train))
print("Validation tree f1_score: ", f1_score(y_valid, y_pred_tree))
print("Validation tree accuracy_score: ", accuracy_score(y_valid, y_pred_tree))
print('*' * 50)
print("Train сat_Boost f1_score: ", f1_score(y_train, y_pred_сat_Boost_train))
print("Train сat_Boost accuracy_score: ", accuracy_score(y_train, y_pred_сat_Boost_train))
print("Validation сat_Boost f1_score: ", f1_score(y_valid, y_pred_сat_Boost))
print("Validation сat_Boost accuracy_score: ", accuracy_score(y_valid, y_pred_сat_Boost))
print('*' * 50)
print("Train LGBM f1_score: ", f1_score(y_train, y_pred_LGBM_train))
print("Train LGBM accuracy_score: ", accuracy_score(y_train, y_pred_LGBM_train))
print("Validation LGBM f1_score: ", f1_score(y_valid, y_pred_LGBM))
print("Validation LGBM accuracy_score: ", accuracy_score(y_valid, y_pred_LGBM))
print('*' * 50)
print("Train xgb f1_score: ", f1_score(y_train, y_pred_xgb_train))
print("Train xgb accuracy_score: ", accuracy_score(y_train, y_pred_xgb_train))
print("Validation xgb f1_score: ", f1_score(y_valid, y_pred_xgb))
print("Validation xgb accuracy_score: ", accuracy_score(y_valid, y_pred_xgb))

/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Train tree f1_score:  0.9468354430379747
Train tree accuracy_score:  0.9473684210526315
Validation tree f1_score:  0.6129032258064515
Validation tree accuracy_score:  0.64
**************************************************
Train сat_Boost f1_score:  0.9333333333333333
Train сat_Boost accuracy_score:  0.9323308270676691
Validation сat_Boost f1_score:  0.7150259067357513
Validation сat_Boost accuracy_score:  0.725
**************************************************
Train LGBM f1_score:  0.9478908188585607
Train LGBM accuracy_score:  0.9473684210526315
Validation LGBM f1_score:  0.6839378238341969
Validation LGBM accuracy_score:  0.695
**************************************************
Train xgb f1_score:  0.948529411764706
Train xgb accuracy_score:  0.9473684210526315
Validation xgb f1_score:  0.6871794871794872
Validation xgb accuracy_score:  0.695


Из приведенных данных выше мы видим среднее качество моделей, а также переобучение. Наилучшие результаты показали модели CatBoostClassifier, LGBM и XGBClassifier. С ними и продолжим работать.

# Подбор признаков, их анализ и оценка важности.

**CatBoost**

In [11]:
col = x_train.columns.tolist()
estimator_cat_boost = PermutationImportance(
    model, scoring= "f1", random_state=27
)
importance_cat_boost = estimator_cat_boost.fit(x_train, y_train)

In [12]:
show_weights(importance_cat_boost, feature_names=col, top=x_train.shape[1])

Weight,Feature
0.0244 ± 0.0122,BCUT2D_MRLOW
0.0159 ± 0.0068,EState_VSA4
0.0143 ± 0.0084,PEOE_VSA6
0.0128 ± 0.0070,NumHDonors
0.0090 ± 0.0068,EState_VSA5
0.0085 ± 0.0083,MolLogP
0.0058 ± 0.0030,Chi0n
0.0052 ± 0.0033,PEOE_VSA8
0.0045 ± 0.0019,EState_VSA7
0.0041 ± 0.0042,VSA_EState4


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [13]:
feature_cat_boost = ['BCUT2D_MRLOW','EState_VSA4','PEOE_VSA6','NumHDonors','EState_VSA5','MolLogP','Chi0n','PEOE_VSA8',
'EState_VSA7','VSA_EState4','PEOE_VSA7','BCUT2D_LOGPHI','AvgIpc','MinAbsEStateIndex','PEOE_VSA9','VSA_EState7',
'SMR_VSA10','EState_VSA3','SlogP_VSA5','SlogP_VSA3','SlogP_VSA12','FractionCSP3','PEOE_VSA10','EState_VSA2',
'BCUT2D_MRHI','VSA_EState9','BCUT2D_LOGPLOW','NumRotatableBonds','BalabanJ','fr_NH0','MinPartialCharge',
'VSA_EState5','FpDensityMorgan2','fr_NH1','PEOE_VSA1','BCUT2D_CHGLO','fr_aryl_methyl','SlogP_VSA1','MaxEStateIndex',
'RingCount','SMR_VSA3','fr_ketone_Topliss','PEOE_VSA11','qed','HallKierAlpha','MaxPartialCharge','NumAliphaticRings',
'SMR_VSA1','VSA_EState8','Ipc','EState_VSA6','fr_alkyl_halide','fr_para_hydroxylation','fr_aldehyde','fr_furan',
'fr_Ar_N','NumAromaticRings','fr_Ar_OH','SlogP_VSA2','NumAliphaticCarbocycles','VSA_EState10','MinEStateIndex',
'EState_VSA8','SMR_VSA7','PEOE_VSA5','fr_bicyclic','SlogP_VSA11','NumSaturatedRings','BCUT2D_CHGHI','PEOE_VSA3',
'fr_aniline','NumAromaticHeterocycles','EState_VSA1','PEOE_VSA12','SlogP_VSA8','SMR_VSA5','EState_VSA9','fr_ester',
'fr_C_O','fr_quatN','fr_thiazole','SlogP_VSA4','PEOE_VSA4','fr_methoxy','fr_Al_COO','fr_ArN','PEOE_VSA13','fr_allylic_oxid',
'EState_VSA10','NumAliphaticHeterocycles','fr_ether','SlogP_VSA10','SPS','SMR_VSA6','SMR_VSA2','PEOE_VSA14','PEOE_VSA2',
'fr_urea','SlogP_VSA7','fr_hdrzone','fr_imide','fr_ketone','fr_lactone','fr_morpholine','fr_nitrile','fr_nitro',
'fr_nitro_arom','fr_oxazole','fr_oxime','fr_piperdine','fr_piperzine','fr_priamide','fr_pyridine','fr_sulfide','fr_sulfonamd',
'fr_sulfone','fr_term_acetylene','fr_tetrazole','fr_thiophene','EState_VSA11','fr_imidazole','fr_hdrzine','fr_NH2',
'VSA_EState1','fr_unbrch_alkane','fr_Al_OH','fr_Ar_COO','fr_Ar_NH','fr_C_S','fr_HOCCN','fr_Imine','fr_Ndealkylation1',
'fr_Ndealkylation2','fr_alkyl_carbamate','fr_amide','fr_azo','NOCount']

In [14]:
target = df['IC50, mM']
data_cb = data[feature_cat_boost]

In [15]:
x_train_cb, x_valid_cb = train_test_split(
    data_cb, train_size=0.8, random_state=1
)
y_train_cb, y_valid_cb = train_test_split(
    target, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_cb.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_cb.shape))

x_train.shape = 798 rows, 137 cols
x_valid.shape = 200 rows, 137 cols


In [16]:
# Создадим модель CatBoost
model = CatBoostClassifier(random_state=27)
# обучение моделей
model.fit(x_train_cb, y_train_cb, verbose=0)
# предсказание ответов для тренеровочной выборки
y_pred_сat_Boost_train = model.predict(x_train_cb)
# предсказание ответов для тестовой выборки
y_pred_сat_Boost = model.predict(x_valid_cb)


print("Train сat_Boost f1_score: ", f1_score(y_train_cb, y_pred_сat_Boost_train))
print("Train сat_Boost accuracy_score: ", accuracy_score(y_train_cb, y_pred_сat_Boost_train))
print("Validation сat_Boost f1_score: ", f1_score(y_valid_cb, y_pred_сat_Boost))
print("Validation сat_Boost accuracy_score: ", accuracy_score(y_valid_cb, y_pred_сat_Boost))


Train сat_Boost f1_score:  0.9295426452410382
Train сat_Boost accuracy_score:  0.9285714285714286
Validation сat_Boost f1_score:  0.7216494845360824
Validation сat_Boost accuracy_score:  0.73


Отобрав признаки мы улучшили метрики для модели LGBMClassifier на вадидации f1_score: 0.72, accuracy_score: 0.73.

**LGBM**

In [17]:
col = x_train.columns.tolist()
estimator_LGBM = PermutationImportance(
    model_LGBM, scoring= "f1", random_state=27
)
importance_LGBM = estimator_LGBM.fit(x_train, y_train)


In [18]:
show_weights(importance_LGBM, feature_names=col, top=x_train.shape[1])

Weight,Feature
0.0197 ± 0.0078,BCUT2D_MRLOW
0.0120 ± 0.0084,NumHDonors
0.0071 ± 0.0082,VSA_EState8
0.0068 ± 0.0076,PEOE_VSA6
0.0047 ± 0.0071,MolLogP
0.0043 ± 0.0066,EState_VSA4
0.0036 ± 0.0044,MinAbsEStateIndex
0.0033 ± 0.0035,VSA_EState5
0.0033 ± 0.0085,MinPartialCharge
0.0032 ± 0.0074,EState_VSA8


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [19]:
feature_LGBM = ['BCUT2D_MRLOW','NumHDonors','VSA_EState8','PEOE_VSA6','MolLogP','EState_VSA4','MinAbsEStateIndex',
'VSA_EState5','MinPartialCharge','EState_VSA8','SMR_VSA10','FpDensityMorgan2','PEOE_VSA8','VSA_EState1','SlogP_VSA5',
'EState_VSA3','BCUT2D_LOGPHI','EState_VSA6','BalabanJ','fr_ether','qed','SMR_VSA1','fr_NH1','Ipc','RingCount','AvgIpc',
'PEOE_VSA10','VSA_EState9','PEOE_VSA2','EState_VSA9','PEOE_VSA1','SMR_VSA6','fr_NH2','MinEStateIndex','SlogP_VSA3',
'NumAliphaticHeterocycles','SlogP_VSA1','fr_aryl_methyl','PEOE_VSA11','EState_VSA2','Chi0n','BCUT2D_MWLOW','PEOE_VSA9',
'fr_C_O','EState_VSA1','EState_VSA5','NumAliphaticRings','BCUT2D_MRHI','fr_aniline','NumSaturatedRings','NumAromaticRings',
'NumAromaticHeterocycles','PEOE_VSA12','fr_Ar_N','fr_para_hydroxylation','fr_Imine','PEOE_VSA13','fr_amide',
'PEOE_VSA14','EState_VSA10','SMR_VSA7','VSA_EState4']

In [20]:
target_LGBM = df['IC50, mM']
data_LGBM = data[feature_LGBM]

In [21]:
x_train_LGBM, x_valid_LGBM = train_test_split(
    data_LGBM, train_size=0.8, random_state=1
)
y_train_LGBM, y_valid_LGBM = train_test_split(
    target_LGBM, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_LGBM.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_LGBM.shape))

x_train.shape = 798 rows, 62 cols
x_valid.shape = 200 rows, 62 cols


In [22]:
model_LGBM = LGBMClassifier(random_state=27)
model_LGBM.fit(x_train_LGBM, y_train_LGBM)
# предсказание ответов для тренеровочной выборки
y_pred_LGBM_train = model_LGBM.predict(x_train_LGBM)
# предсказание ответов для тестовой выборки
y_pred_LGBM = model_LGBM.predict(x_valid_LGBM)

print("Train LGBM f1_score: ", f1_score(y_train_LGBM, y_pred_LGBM_train))
print("Train LGBM accuracy_score: ", accuracy_score(y_train_LGBM, y_pred_LGBM_train))
print("Validation LGBM f1_score: ", f1_score(y_valid_LGBM, y_pred_LGBM))
print("Validation LGBM accuracy_score: ", accuracy_score(y_valid_LGBM, y_pred_LGBM))

Train LGBM f1_score:  0.9482758620689655
Train LGBM accuracy_score:  0.9473684210526315
Validation LGBM f1_score:  0.6842105263157895
Validation LGBM accuracy_score:  0.7


Отобрав признаки мы немного улучшили метрики для модели LGBMClassifier на вадидации f1_score: 0.684, accuracy_score: 0.7.

**xgboost**

In [23]:
col = x_train.columns.tolist()
estimator_xgb = PermutationImportance(
    xgb_r, scoring= "neg_mean_squared_error", random_state=27
)
importance_xgb = estimator_xgb.fit(x_train, y_train)

In [24]:
show_weights(importance_xgb, feature_names=col, top=x_train.shape[1])

Weight,Feature
0.0211 ± 0.0084,BCUT2D_MRLOW
0.0185 ± 0.0040,NumHDonors
0.0065 ± 0.0033,MolLogP
0.0058 ± 0.0088,VSA_EState8
0.0045 ± 0.0063,fr_NH1
0.0043 ± 0.0034,SlogP_VSA5
0.0038 ± 0.0069,EState_VSA6
0.0035 ± 0.0073,BCUT2D_LOGPHI
0.0035 ± 0.0080,Ipc
0.0035 ± 0.0073,EState_VSA2


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [25]:
feature_xgb = ['BCUT2D_MRLOW','NumHDonors','MolLogP','VSA_EState8','fr_NH1','SlogP_VSA5','EState_VSA6','BCUT2D_LOGPHI',
'Ipc','EState_VSA2','PEOE_VSA6','FpDensityMorgan2','AvgIpc','EState_VSA4','VSA_EState5','MaxEStateIndex','SlogP_VSA12',
'MinAbsEStateIndex','qed','VSA_EState7','NumSaturatedHeterocycles','NumRotatableBonds','VSA_EState1','PEOE_VSA8',
'HallKierAlpha','PEOE_VSA10','SMR_VSA7','EState_VSA3','EState_VSA10','EState_VSA9','SMR_VSA10','SPS','VSA_EState9',
'NOCount','MinPartialCharge','PEOE_VSA12','PEOE_VSA1','PEOE_VSA11','BCUT2D_LOGPLOW','fr_pyridine','NumAromaticHeterocycles',
'NumAliphaticRings','SMR_VSA1','PEOE_VSA9','fr_allylic_oxid','fr_para_hydroxylation','EState_VSA5','FractionCSP3', 
'MinEStateIndex','fr_ester',]

In [26]:
target_xgb = df['IC50, mM']
data_xgb = data[feature_xgb]

In [27]:
x_train_xgb, x_valid_xgb = train_test_split(
    data_xgb, train_size=0.8, random_state=1
)
y_train_xgb, y_valid_xgb = train_test_split(
    target_xgb, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_xgb.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_xgb.shape))

x_train.shape = 798 rows, 50 cols
x_valid.shape = 200 rows, 50 cols


In [28]:
model_xgb = xg.XGBClassifier(random_state=27)
model_xgb.fit(x_train_xgb, y_train_xgb, verbose=100)
y_pred_xgb = model_xgb.predict(x_train_xgb)
y_val_xgb = model_xgb.predict(x_valid_xgb)
print("Train f1_score: ", f1_score(y_train_xgb, y_pred_xgb))
print("Train accuracy_score: ", accuracy_score(y_train_xgb, y_pred_xgb))
print("Validation f1_score: ", f1_score(y_valid_xgb, y_val_xgb))
print("Validation accuracy_score: ", accuracy_score(y_valid_xgb, y_val_xgb))

Train f1_score:  0.948019801980198
Train accuracy_score:  0.9473684210526315
Validation f1_score:  0.690721649484536
Validation accuracy_score:  0.7


Отобрав признаки мы улучшили метрики для модели xgboost на вадидации f1_score: 0.69, accuracy_score: 0.7.

# Подбор гиперпараметров

**CatBoost**

In [29]:
def catboost_cv(depth, learning_rate,n_estimators):
    depth = int(depth)
    n_estimators = int(n_estimators)
    model = CatBoostClassifier(
        depth=depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        eval_metric  = 'F1',
        random_seed=27
    )
    model.fit(x_train_cb, y_train_cb, verbose=100)

    return f1_score(y_valid_cb, model.predict(x_valid_cb))

In [30]:
param_space = {
    'depth': (2, 10),             
    'learning_rate': (0.001, 0.3),
    'n_estimators': (50, 1500)
}

bayesian_opt = BayesianOptimization(
    f=catboost_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=25)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   |   depth   | learni... | n_esti... |
-------------------------------------------------------------
0:	learn: 0.7045952	total: 21ms	remaining: 23.4s
100:	learn: 0.9477612	total: 523ms	remaining: 5.25s
200:	learn: 0.9475000	total: 986ms	remaining: 4.49s
300:	learn: 0.9485294	total: 1.45s	remaining: 3.94s
400:	learn: 0.9477612	total: 1.91s	remaining: 3.4s
500:	learn: 0.9480198	total: 2.36s	remaining: 2.9s
600:	learn: 0.9484029	total: 2.81s	remaining: 2.41s
700:	learn: 0.9480198	total: 3.26s	remaining: 1.93s
800:	learn: 0.9475000	total: 3.72s	remaining: 1.46s
900:	learn: 0.9482759	total: 4.16s	remaining: 994ms
1000:	learn: 0.9484029	total: 4.61s	remaining: 530ms
1100:	learn: 0.9478908	total: 5.06s	remaining: 69ms
1115:	learn: 0.9478908	total: 5.13s	remaining: 0us
| 1         | 0.6839    | 5.406     | 0.2446    | 1.116e+03 |
0:	learn: 0.7065593	total: 39.4ms	remaining: 57.8s
100:	learn: 0.9481481	total: 3.12s	remaining: 42.2s
200:	learn: 0.9480198	total: 6.06s	remai

400:	learn: 0.9476309	total: 1.82s	remaining: 4.85s
500:	learn: 0.9477612	total: 2.28s	remaining: 4.39s
600:	learn: 0.9477612	total: 2.73s	remaining: 3.93s
700:	learn: 0.9480198	total: 3.18s	remaining: 3.47s
800:	learn: 0.9482759	total: 3.64s	remaining: 3.02s
900:	learn: 0.9484029	total: 4.09s	remaining: 2.57s
1000:	learn: 0.9480198	total: 4.54s	remaining: 2.11s
1100:	learn: 0.9480198	total: 4.99s	remaining: 1.66s
1200:	learn: 0.9481481	total: 5.43s	remaining: 1.2s
1300:	learn: 0.9476309	total: 5.88s	remaining: 751ms
1400:	learn: 0.9478908	total: 6.33s	remaining: 298ms
1466:	learn: 0.9478908	total: 6.62s	remaining: 0us
| 11        | 0.6875    | 5.73      | 0.1847    | 1.468e+03 |
0:	learn: 0.6962025	total: 88.9ms	remaining: 1m 39s
100:	learn: 0.9075216	total: 5.62s	remaining: 56.8s
200:	learn: 0.9333333	total: 11.1s	remaining: 50.8s
300:	learn: 0.9397294	total: 16.5s	remaining: 45s
400:	learn: 0.9443758	total: 22.1s	remaining: 39.7s
500:	learn: 0.9481481	total: 28.3s	remaining: 35.1s
6

1000:	learn: 0.9482759	total: 14.5s	remaining: 6.58s
1100:	learn: 0.9482759	total: 15.9s	remaining: 5.13s
1200:	learn: 0.9478908	total: 17.3s	remaining: 3.67s
1300:	learn: 0.9480198	total: 18.7s	remaining: 2.22s
1400:	learn: 0.9481481	total: 20.1s	remaining: 776ms
1454:	learn: 0.9477612	total: 20.9s	remaining: 0us
| 20        | 0.701     | 7.944     | 0.04392   | 1.455e+03 |
0:	learn: 0.6962025	total: 66.8ms	remaining: 1m 37s
100:	learn: 0.9075216	total: 5.65s	remaining: 1m 16s
200:	learn: 0.9323493	total: 11.3s	remaining: 1m 10s
300:	learn: 0.9398773	total: 17s	remaining: 1m 5s
400:	learn: 0.9468480	total: 22.6s	remaining: 59.8s
500:	learn: 0.9478908	total: 28.3s	remaining: 54.2s
600:	learn: 0.9478908	total: 33.9s	remaining: 48.5s
700:	learn: 0.9480198	total: 39.5s	remaining: 42.9s
800:	learn: 0.9480198	total: 45.1s	remaining: 37.2s
900:	learn: 0.9480198	total: 50.7s	remaining: 31.6s
1000:	learn: 0.9480198	total: 56.3s	remaining: 25.9s
1100:	learn: 0.9480198	total: 1m 1s	remaining: 20

400:	learn: 0.9481481	total: 23s	remaining: 41.4s
500:	learn: 0.9480198	total: 28.6s	remaining: 35.5s
600:	learn: 0.9481481	total: 34.3s	remaining: 29.8s
700:	learn: 0.9480198	total: 39.9s	remaining: 24s
800:	learn: 0.9476309	total: 45.6s	remaining: 18.3s
900:	learn: 0.9482759	total: 51.2s	remaining: 12.6s
1000:	learn: 0.9480198	total: 56.9s	remaining: 6.93s
1100:	learn: 0.9482759	total: 1m 2s	remaining: 1.25s
1122:	learn: 0.9478908	total: 1m 3s	remaining: 0us
| 30        | 0.6974    | 9.093     | 0.1307    | 1.123e+03 |


In [31]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'l2_leaf_reg'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'depth': 9, 'learning_rate': 0.053803928959764186, 'n_estimators': 1119.1458022388763}
Best R-squared Score: 0.7208


In [33]:
params = {
    'depth': 9,             
    'learning_rate': 0.053803928959764186, 
    'n_estimators': 1119
}
model = CatBoostClassifier(**params, random_state=27)
model.fit(x_train_cb, y_train_cb, verbose=100)
y_pred_cb = model.predict(x_train_cb)
y_val_cb = model.predict(x_valid_cb)
print("Train сat_Boost f1_score: ", f1_score(y_train_cb, y_pred_cb))
print("Train сat_Boost accuracy_score: ", accuracy_score(y_train_cb, y_pred_cb))
print("Validation сat_Boost f1_score: ", f1_score(y_valid_cb, y_val_cb))
print("Validation сat_Boost accuracy_score: ", accuracy_score(y_valid_cb, y_val_cb))

0:	learn: 0.6770399	total: 60.4ms	remaining: 1m 7s
100:	learn: 0.2455168	total: 5.68s	remaining: 57.3s
200:	learn: 0.1660394	total: 11.2s	remaining: 51.4s
300:	learn: 0.1274099	total: 16.9s	remaining: 45.9s
400:	learn: 0.1119937	total: 22.6s	remaining: 40.4s
500:	learn: 0.1049801	total: 28.4s	remaining: 35s
600:	learn: 0.1007741	total: 34s	remaining: 29.3s
700:	learn: 0.0982665	total: 39.6s	remaining: 23.6s
800:	learn: 0.0963972	total: 45.2s	remaining: 17.9s
900:	learn: 0.0951578	total: 50.9s	remaining: 12.3s
1000:	learn: 0.0941796	total: 56.5s	remaining: 6.66s
1100:	learn: 0.0934322	total: 1m 2s	remaining: 1.02s
1118:	learn: 0.0933310	total: 1m 3s	remaining: 0us
Train сat_Boost f1_score:  0.9477611940298508
Train сat_Boost accuracy_score:  0.9473684210526315
Validation сat_Boost f1_score:  0.7208121827411168
Validation сat_Boost accuracy_score:  0.725


После подбора гиперпараметров мы получили метрику модели CatBoost на вадидации f1_score: 0.72, accuracy_score: 0.725.

**LGBM**

In [34]:
def LGBM_cv(max_depth, learning_rate, n_estimators):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    model = LGBMClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,          
        eval_metric  = 'F1',
        random_seed=27
    )
    model.fit(x_train_LGBM, y_train_LGBM, eval_set = (x_valid_LGBM, y_valid_LGBM))
    return f1_score(y_valid_LGBM, model.predict(x_valid_LGBM))

In [35]:
param_space = {
    'max_depth': (2, 10),            
    'learning_rate': (0.001, 0.3),
    'n_estimators': (50, 2000)
}

bayesian_opt = BayesianOptimization(
    f=LGBM_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=50)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
[LightGBM] [Warning] Unknown parameter: eval_metric
[1]	valid_0's binary_logloss: 0.659389
[2]	valid_0's binary_logloss: 0.633887
[3]	valid_0's binary_logloss: 0.618589
[4]	valid_0's binary_logloss: 0.609455
[5]	valid_0's binary_logloss: 0.596191
[6]	valid_0's binary_logloss: 0.591258
[7]	valid_0's binary_logloss: 0.586247
[8]	valid_0's binary_logloss: 0.581994
[9]	valid_0's binary_logloss: 0.578123
[10]	valid_0's binary_logloss: 0.579825
[11]	valid_0's binary_logloss: 0.582568
[12]	valid_0's binary_logloss: 0.584508
[13]	valid_0's binary_logloss: 0.584432
[14]	valid_0's binary_logloss: 0.584341
[15]	valid_0's binary_logloss: 0.586897
[16]	valid_0's binary_logloss: 0.589761
[17]	valid_0's binary_logloss: 0.593326
[18]	valid_0's binary_logloss: 0.592355
[19]	valid_0's binary_logloss: 0.595079
[20]	valid_0's binary_logloss: 0.597215
[21]	valid_0's binary_logloss: 0.

[290]	valid_0's binary_logloss: 1.28452
[291]	valid_0's binary_logloss: 1.28469
[292]	valid_0's binary_logloss: 1.28673
[293]	valid_0's binary_logloss: 1.28932
[294]	valid_0's binary_logloss: 1.29011
[295]	valid_0's binary_logloss: 1.29228
[296]	valid_0's binary_logloss: 1.29343
[297]	valid_0's binary_logloss: 1.29585
[298]	valid_0's binary_logloss: 1.29829
[299]	valid_0's binary_logloss: 1.29793
[300]	valid_0's binary_logloss: 1.29747
[301]	valid_0's binary_logloss: 1.2974
[302]	valid_0's binary_logloss: 1.29992
[303]	valid_0's binary_logloss: 1.29912
[304]	valid_0's binary_logloss: 1.2972
[305]	valid_0's binary_logloss: 1.29703
[306]	valid_0's binary_logloss: 1.29741
[307]	valid_0's binary_logloss: 1.29935
[308]	valid_0's binary_logloss: 1.29806
[309]	valid_0's binary_logloss: 1.2996
[310]	valid_0's binary_logloss: 1.30075
[311]	valid_0's binary_logloss: 1.3033
[312]	valid_0's binary_logloss: 1.30274
[313]	valid_0's binary_logloss: 1.30478
[314]	valid_0's binary_logloss: 1.3088
[315]

[591]	valid_0's binary_logloss: 1.58838
[592]	valid_0's binary_logloss: 1.59018
[593]	valid_0's binary_logloss: 1.59041
[594]	valid_0's binary_logloss: 1.59071
[595]	valid_0's binary_logloss: 1.59102
[596]	valid_0's binary_logloss: 1.59258
[597]	valid_0's binary_logloss: 1.59294
[598]	valid_0's binary_logloss: 1.59315
[599]	valid_0's binary_logloss: 1.59312
[600]	valid_0's binary_logloss: 1.59333
[601]	valid_0's binary_logloss: 1.59335
[602]	valid_0's binary_logloss: 1.59361
[603]	valid_0's binary_logloss: 1.59399
[604]	valid_0's binary_logloss: 1.59448
[605]	valid_0's binary_logloss: 1.59507
[606]	valid_0's binary_logloss: 1.59501
[607]	valid_0's binary_logloss: 1.59555
[608]	valid_0's binary_logloss: 1.59575
[609]	valid_0's binary_logloss: 1.59589
[610]	valid_0's binary_logloss: 1.5959
[611]	valid_0's binary_logloss: 1.59592
[612]	valid_0's binary_logloss: 1.59569
[613]	valid_0's binary_logloss: 1.596
[614]	valid_0's binary_logloss: 1.5966
[615]	valid_0's binary_logloss: 1.59704
[616

[831]	valid_0's binary_logloss: 1.66076
[832]	valid_0's binary_logloss: 1.66085
[833]	valid_0's binary_logloss: 1.66105
[834]	valid_0's binary_logloss: 1.66184
[835]	valid_0's binary_logloss: 1.66194
[836]	valid_0's binary_logloss: 1.66218
[837]	valid_0's binary_logloss: 1.66237
[838]	valid_0's binary_logloss: 1.66257
[839]	valid_0's binary_logloss: 1.66265
[840]	valid_0's binary_logloss: 1.66281
[841]	valid_0's binary_logloss: 1.6629
[842]	valid_0's binary_logloss: 1.663
[843]	valid_0's binary_logloss: 1.66385
[844]	valid_0's binary_logloss: 1.66506
[845]	valid_0's binary_logloss: 1.66521
[846]	valid_0's binary_logloss: 1.66609
[847]	valid_0's binary_logloss: 1.66612
[848]	valid_0's binary_logloss: 1.66624
[849]	valid_0's binary_logloss: 1.6663
[850]	valid_0's binary_logloss: 1.66643
[851]	valid_0's binary_logloss: 1.66648
[852]	valid_0's binary_logloss: 1.66664
[853]	valid_0's binary_logloss: 1.6669
[854]	valid_0's binary_logloss: 1.66678
[855]	valid_0's binary_logloss: 1.66695
[856]

[1126]	valid_0's binary_logloss: 1.7034
[1127]	valid_0's binary_logloss: 1.70351
[1128]	valid_0's binary_logloss: 1.70361
[1129]	valid_0's binary_logloss: 1.70373
[1130]	valid_0's binary_logloss: 1.70379
[1131]	valid_0's binary_logloss: 1.70395
[1132]	valid_0's binary_logloss: 1.70402
[1133]	valid_0's binary_logloss: 1.70412
[1134]	valid_0's binary_logloss: 1.70435
[1135]	valid_0's binary_logloss: 1.70454
[1136]	valid_0's binary_logloss: 1.70466
[1137]	valid_0's binary_logloss: 1.70489
[1138]	valid_0's binary_logloss: 1.70503
[1139]	valid_0's binary_logloss: 1.70514
[1140]	valid_0's binary_logloss: 1.70525
[1141]	valid_0's binary_logloss: 1.70525
[1142]	valid_0's binary_logloss: 1.70529
[1143]	valid_0's binary_logloss: 1.70544
[1144]	valid_0's binary_logloss: 1.70552
[1145]	valid_0's binary_logloss: 1.70569
[1146]	valid_0's binary_logloss: 1.70589
[1147]	valid_0's binary_logloss: 1.70604
[1148]	valid_0's binary_logloss: 1.70616
[1149]	valid_0's binary_logloss: 1.70619
[1150]	valid_0's 

...

[391]	valid_0's binary_logloss: 0.611608
[392]	valid_0's binary_logloss: 0.612185
[393]	valid_0's binary_logloss: 0.612754
[394]	valid_0's binary_logloss: 0.613838
[395]	valid_0's binary_logloss: 0.6141
[396]	valid_0's binary_logloss: 0.614591
[397]	valid_0's binary_logloss: 0.615003
[398]	valid_0's binary_logloss: 0.614949
[399]	valid_0's binary_logloss: 0.615908
[400]	valid_0's binary_logloss: 0.615811
[401]	valid_0's binary_logloss: 0.61652
[402]	valid_0's binary_logloss: 0.616839
[403]	valid_0's binary_logloss: 0.617477
[404]	valid_0's binary_logloss: 0.617924
[405]	valid_0's binary_logloss: 0.617203
[406]	valid_0's binary_logloss: 0.617605
[407]	valid_0's binary_logloss: 0.618188
[408]	valid_0's binary_logloss: 0.618388
[409]	valid_0's binary_logloss: 0.618315
[410]	valid_0's binary_logloss: 0.618585
[411]	valid_0's binary_logloss: 0.619293
[412]	valid_0's binary_logloss: 0.619139
[413]	valid_0's binary_logloss: 0.619695
[414]	valid_0's binary_logloss: 0.620257
[415]	valid_0's bin

[830]	valid_0's binary_logloss: 0.742906
[831]	valid_0's binary_logloss: 0.74352
[832]	valid_0's binary_logloss: 0.743876
[833]	valid_0's binary_logloss: 0.744885
[834]	valid_0's binary_logloss: 0.745203
[835]	valid_0's binary_logloss: 0.74542
[836]	valid_0's binary_logloss: 0.745632
[837]	valid_0's binary_logloss: 0.745884
[838]	valid_0's binary_logloss: 0.74621
[839]	valid_0's binary_logloss: 0.746278
[840]	valid_0's binary_logloss: 0.747067
[841]	valid_0's binary_logloss: 0.747444
[842]	valid_0's binary_logloss: 0.748193
[843]	valid_0's binary_logloss: 0.748124
[844]	valid_0's binary_logloss: 0.748503
[845]	valid_0's binary_logloss: 0.748456
[846]	valid_0's binary_logloss: 0.748487
[847]	valid_0's binary_logloss: 0.748823
[848]	valid_0's binary_logloss: 0.750002
[849]	valid_0's binary_logloss: 0.750767
[850]	valid_0's binary_logloss: 0.751794
[851]	valid_0's binary_logloss: 0.752353
[852]	valid_0's binary_logloss: 0.752112
[853]	valid_0's binary_logloss: 0.752731
[854]	valid_0's bin

[1246]	valid_0's binary_logloss: 0.84413
[1247]	valid_0's binary_logloss: 0.845018
[1248]	valid_0's binary_logloss: 0.84539
[1249]	valid_0's binary_logloss: 0.845783
[1250]	valid_0's binary_logloss: 0.846112
[1251]	valid_0's binary_logloss: 0.845548
[1252]	valid_0's binary_logloss: 0.845881
[1253]	valid_0's binary_logloss: 0.846444
[1254]	valid_0's binary_logloss: 0.847201
[1255]	valid_0's binary_logloss: 0.847184
[1256]	valid_0's binary_logloss: 0.847286
[1257]	valid_0's binary_logloss: 0.847745
[1258]	valid_0's binary_logloss: 0.848119
[1259]	valid_0's binary_logloss: 0.849562
[1260]	valid_0's binary_logloss: 0.850206
[1261]	valid_0's binary_logloss: 0.850723
[1262]	valid_0's binary_logloss: 0.850332
[1263]	valid_0's binary_logloss: 0.850368
[1264]	valid_0's binary_logloss: 0.850191
[1265]	valid_0's binary_logloss: 0.850593
[1266]	valid_0's binary_logloss: 0.850427
[1267]	valid_0's binary_logloss: 0.850372
[1268]	valid_0's binary_logloss: 0.850721
[1269]	valid_0's binary_logloss: 0.8

[1667]	valid_0's binary_logloss: 0.946459
[1668]	valid_0's binary_logloss: 0.94627
[1669]	valid_0's binary_logloss: 0.946346
[1670]	valid_0's binary_logloss: 0.946185
[1671]	valid_0's binary_logloss: 0.946391
[1672]	valid_0's binary_logloss: 0.946079
[1673]	valid_0's binary_logloss: 0.94615
[1674]	valid_0's binary_logloss: 0.946599
[1675]	valid_0's binary_logloss: 0.946484
[1676]	valid_0's binary_logloss: 0.946912
[1677]	valid_0's binary_logloss: 0.946825
[1678]	valid_0's binary_logloss: 0.947208
[1679]	valid_0's binary_logloss: 0.947023
[1680]	valid_0's binary_logloss: 0.946574
[1681]	valid_0's binary_logloss: 0.946653
[1682]	valid_0's binary_logloss: 0.947361
[1683]	valid_0's binary_logloss: 0.947691
[1684]	valid_0's binary_logloss: 0.947341
[1685]	valid_0's binary_logloss: 0.947654
[1686]	valid_0's binary_logloss: 0.947661
[1687]	valid_0's binary_logloss: 0.948331
[1688]	valid_0's binary_logloss: 0.948659
[1689]	valid_0's binary_logloss: 0.948893
[1690]	valid_0's binary_logloss: 0.9

[288]	valid_0's binary_logloss: 0.57717
[289]	valid_0's binary_logloss: 0.577093
[290]	valid_0's binary_logloss: 0.576907
[291]	valid_0's binary_logloss: 0.576668
[292]	valid_0's binary_logloss: 0.576643
[293]	valid_0's binary_logloss: 0.576704
[294]	valid_0's binary_logloss: 0.576772
[295]	valid_0's binary_logloss: 0.576732
[296]	valid_0's binary_logloss: 0.576751
[297]	valid_0's binary_logloss: 0.576689
[298]	valid_0's binary_logloss: 0.576615
[299]	valid_0's binary_logloss: 0.576544
[300]	valid_0's binary_logloss: 0.576565
[301]	valid_0's binary_logloss: 0.576506
[302]	valid_0's binary_logloss: 0.576503
[303]	valid_0's binary_logloss: 0.576431
[304]	valid_0's binary_logloss: 0.576241
[305]	valid_0's binary_logloss: 0.576013
[306]	valid_0's binary_logloss: 0.575939
[307]	valid_0's binary_logloss: 0.575765
[308]	valid_0's binary_logloss: 0.575836
[309]	valid_0's binary_logloss: 0.575823
[310]	valid_0's binary_logloss: 0.575768
[311]	valid_0's binary_logloss: 0.575739
[312]	valid_0's b

In [36]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'n_estimators'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'learning_rate': 0.271434557695652, 'max_depth': 8.774433598643647, 'n_estimators': 318}
Best R-squared Score: 0.6904


In [37]:
params = {
    'max_depth': 8,            
    'learning_rate': 0.271434557695652,  
    'n_estimators': 318
}

model_LGBM = LGBMClassifier(**params, random_state=27)
model_LGBM.fit(x_train_LGBM, y_train_LGBM)
# предсказание ответов для тренеровочной выборки
y_pred_LGBM_train = model_LGBM.predict(x_train_LGBM)
# предсказание ответов для тестовой выборки
y_pred_LGBM = model_LGBM.predict(x_valid_LGBM)

print("Train LGBM f1_score: ", f1_score(y_train_LGBM, y_pred_LGBM_train))
print("Train LGBM accuracy_score: ", accuracy_score(y_train_LGBM, y_pred_LGBM_train))
print("Validation LGBM f1_score: ", f1_score(y_valid_LGBM, y_pred_LGBM))
print("Validation LGBM accuracy_score: ", accuracy_score(y_valid_LGBM, y_pred_LGBM))

Train LGBM f1_score:  0.9484029484029484
Train LGBM accuracy_score:  0.9473684210526315
Validation LGBM f1_score:  0.6903553299492386
Validation LGBM accuracy_score:  0.695


После подбора гиперпараметров мы улучшили метрики LGBMClassifier на вадидации f1_score: 0.69, accuracy_score:  0.695.

**xgboost**

In [38]:
def XGB_cv(max_depth, learning_rate, n_estimators):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    model = xg.XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        random_seed=27
    )
    model.fit(x_train_xgb, y_train_xgb)
    return f1_score(y_valid_xgb, model.predict(x_valid_xgb))

In [39]:
param_space = {
    'max_depth': (2, 10),            
    'learning_rate': (0.001, 0.3),  
    'n_estimators': (10, 2000)
}

bayesian_opt = BayesianOptimization(
    allow_duplicate_points=True,f=XGB_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=50)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [08:48:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 1         | 0.6771    | 0.1283    | 8.517     | 1.473e+03 |


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [08:48:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 2         | 0.6839    | 0.2605    | 5.067     | 1.959e+03 |


...

| 53        | 0.6492    | 0.0836    | 6.92      | 136.5     |


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [08:49:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 54        | 0.6842    | 0.07239   | 2.473     | 1.808e+03 |


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [08:49:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 55        | 0.6842    | 0.01312   | 2.343     | 697.7     |


In [40]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'n_estimators'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'learning_rate': 0.24156191458213627, 'max_depth': 8.44220920786574, 'n_estimators': 1014}
Best R-squared Score: 0.6974


In [41]:
params = {
    'max_depth': 8,            
    'learning_rate':0.24156191458213627
    ,  
    'n_estimators': 1014
}
model_XGB = xg.XGBClassifier(**params, random_state=27)
model_XGB.fit(x_train_xgb, y_train_xgb)
y_pred_xgb = model_XGB.predict(x_train_xgb)
y_val_xgb = model_XGB.predict(x_valid_xgb)
print("Train f1_score: ", f1_score(y_train_xgb, y_pred_xgb))
print("Train accuracy_score: ", accuracy_score(y_train_xgb, y_pred_xgb))
print("Validation f1_score: ", f1_score(y_valid_xgb, y_val_xgb))
print("Validation accuracy_score: ", accuracy_score(y_valid_xgb, y_val_xgb))

Train f1_score:  0.9481481481481482
Train accuracy_score:  0.9473684210526315
Validation f1_score:  0.6974358974358974
Validation accuracy_score:  0.705


После подбора гиперпараметров мы улучшили метрики XGBClassifier на вадидации f1_score: 0.6974, accuracy_score:  0.705. 

**Выводы**

Мы сравнили 4 регрессионые модели: DecisionTreeRegressor, CatBoostRegressor, LGBMRegressor, XGBRegressor. Наилучший результат показала модель CatBoostClassifier с результатом на трейне: f1_score=  0.95 и accuracy_score= 0.95 и тесте: f1_score=0.72, accuracy_score= 0.725.

**Рекомендации**

Для улучшения значения метрики и снижения переобучения нужно будет увеличить количество данных.